# Music Recommender System using Apache Spark and Python
**Estimated time: 8hrs**

## Description

For this project, you are to create a recommender system that will recommend new musical artists to a user based on their listening history. Suggesting different songs or musical artists to a user is important to many music streaming services, such as Pandora and Spotify. In addition, this type of recommender system could also be used as a means of suggesting TV shows or movies to a user (e.g., Netflix). 

To create this system you will be using Spark and the collaborative filtering technique. The instructions for completing this project will be laid out entirely in this file. You will have to implement any missing code as well as answer any questions.

**Submission Instructions:** 
* Add all of your updates to this IPython file and do not clear any of the output you get from running your code.
* Upload this file onto moodle.

## Datasets

You will be using some publicly available song data from audioscrobbler, which can be found [here](http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). However, we modified the original data files so that the code will run in a reasonable time on a single machine. The reduced data files have been suffixed with `_small.txt` and contains only the information relevant to the top 50 most prolific users (highest artist play counts).

The original data file `user_artist_data.txt` contained about 141,000 unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists are recorded, along with their count.

Note that when plays are scribbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard, and this may only be detected later. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they clearly refer to the same artist. So, the data set includes `artist_alias.txt`, which maps artist IDs that are known misspellings or variants to the canonical ID of that artist.

The `artist_data.txt` file then provides a map from the canonical artist ID to the name of the artist.

## Necessary Package Imports

In [3]:
from pyspark.mllib.recommendation import *
import random
from operator import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

## Loading data

Load the three datasets into RDDs and name them `artistData`, `artistAlias`, and `userArtistData`. View the README, or the files themselves, to see how this data is formated. Some of the files have tab delimeters while some have space delimiters. Make sure that your `userArtistData` RDD contains only the canonical artist IDs.

In [4]:
# artistData is rdd containing artist data, where
# First map is used to split every tuple/line on bases of delimiter: "\t" between each item in the tuple
# Second map is used to convert every item into their respective data types.
# line[0]: ; line[1]:
artistData = sc.textFile("artist_data_small.txt")\
                    .map(lambda line: line.split("\t"))\
                    .map(lambda line: (int(line[0]), line[1]))

# artistAlias is rdd containing artist alias data, where
# First map is used to split every tuple/line on bases of delimiter: "\t" between each item in the tuple
# Second map is used to convert every item into their respective data types.
# line[0]: ; line[1]: 
artistAlias = sc.textFile("artist_alias_small.txt")\
                    .map(lambda line: line.split("\t"))\
                    .map(lambda line: (int(line[0]), int(line[1])))

# artistData is rdd containing artist data, where
# First map is used to split every tuple/line on bases of delimiter: " " between each item in the tuple
# Second map is used to convert every item into their respective data types.
# line[0]: ; line[1]: ; line[2]: 
userArtistData = sc.textFile("user_artist_data_small.txt")\
                        .map(lambda line: line.split(" "))\
                        .map(lambda line: (int(line[0]), int(line[1]), int(line[2])))

# Created artist_alias_info dictionary to store the artist alias in more feasible structure for future use.
artist_alias_info = {}
for line in artistAlias.collect():
    artist_alias_info[line[0]] = line[1]

# Edited userArtistData with only the canonical artist ID
# Lambda function is a if else implementation where is artist ID found in alias dictionary create above, 
# the original ID will be taken and mapped into userArtistData
# x[0] is user ID; x[1] is artist ID/artist alias ID; x[2] is number of play counts of artist
userArtistData = userArtistData\
                .map(lambda x: (x[0], artist_alias_info[x[1]], x[2]) 
                                        if x[1] in artist_alias_info 
                                           else (x[0], x[1], x[2]))

## Data Exploration

In the blank below, write some code that with find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```


In [5]:
# Lambda function is taken to organize the data in a single rdd structure such that 
# it is more manageable for future
# Lambda function to map only two parameters of interest in new rdd structure u[0] is user 
# and u[2] is play count of song
# u[0] is same as x[0] and u[2] is same as x[1] in second lambda statement but we are combining three parameters here.
# First value is user, Second is play count and Third is mean of play count as needed in order of printing.
userPlay = userArtistData.map(lambda u: (u[0], u[2]))\
                .groupByKey().mapValues(list)\
                .map(lambda x: (x[0], sum(x[1]), sum(x[1])/len(x[1])))\
                .sortBy(ascending=False, keyfunc = lambda x: x[1]).take(3)

for line in userPlay:
    print "User %s has a total play count of %s and a mean play count of %s."%(line[0], line[1], line[2])

User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`userArtistData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look as follows:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031
```

In [6]:
trainData, validationData, testData = userArtistData.randomSplit([4, 4, 2], 13)

# Printing the first 3 elements of each data 
print trainData.take(3)
print validationData.take(3)
print testData.take(3)

# Printing the size of data sets
print trainData.count()
print validationData.count()
print testData.count()

# Saving the data in cache.
trainData = trainData.cache()
validationData = validationData.cache()
testData = testData.cache()

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031


## The Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: [http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html). The [function you will be using](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS.trainImplicit) has a few tunable parameters that will affect how the model is built. Therefore, to get the best model, we will do a small parameter sweep and choose the model that performs the best on the validation set

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some dataset of *true* artist plays for a set of users. This model can be used to predict the top X artist recommendations for a user and these recommendations can be compared the artists that the user actually listened to (here, X will be the number of artists in the dataset of *true* artist plays). Then, the fraction of overlap between the top X predictions of the model and the X artists that the user actually listened to can be calculated. This process can be repeated for all users and an average value returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 artists for a user. Suppose, that user actually listened to the artists [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: when using the model to predict the top-X artists for a user, do not include the artists listed with that user in the training data.**

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit) and a dataset as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`).

In [11]:
# modelEval function to evalutate model on basis of validation data
def modelEval(model, dataset):
    global trainData
    
    # Creating a list of Artists such that we can remove the artists from the training data
    artists = (set(userArtistData.map(lambda x:(x[1])).collect()))
    users = (set(dataset.map(lambda x:(x[0])).collect()))
    
    # User-Artist relation in the Dataset 
    user_artist_rel_DataSet = dict(dataset.map(lambda x: (x[0], x[1])).groupByKey().mapValues(set).collect())
    
    # User-Artist relation in the Training Data
    # Both the statements are similar in fashion with exception to the datasets taken to create the dictionary
    user_artist_rel = dict(trainData.map(lambda x:(x[0], x[1])).groupByKey().mapValues(set).collect())
    
    # similarity variable will be required to calculate the score for each user
    # overall score = similarity/ number of users 
    # First map in the model.predictAll statement is used for sorting the tuples over ratings
    # Second map in the mode.predictAll statement is used for fetching all the artists/product 
    similarity = 0
    for user in users:
        original_artist = user_artist_rel_DataSet[user]
        prediction_model = map(lambda x: (user, x), set(artists) - set(user_artist_rel[user]))
        predict_artist = model.predictAll(sc.parallelize(prediction_model))\
                            .sortBy(ascending = False, keyfunc= lambda x : x[2])\
                            .map(lambda x: x[1])\
                            .take(len(original_artist))
        similarity += float(len(set(predict_artist).intersection(original_artist)))/ len(original_artist)
    return float(similarity)/len(users)

### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering) (leave everything else at its default value, **except make `seed`=345**). Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Note: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is 0.090431
The model score for rank 10 is 0.095294
The model score for rank 20 is 0.090248
```

In [12]:
ranks = [2, 10, 20]

for rank in ranks:
        model = ALS.trainImplicit(trainData, rank, seed = 345)
        print "The model score for rank %d is %s"%(rank, str(modelEval(model, validationData)))

The model score for rank 2 is 0.0934617672415
The model score for rank 10 is 0.0978989450986
The model score for rank 20 is 0.0842589842136


Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.0507`.

In [13]:
bestModel = ALS.trainImplicit(trainData, rank=2, seed=345)
modelEval(bestModel, testData)

0.05627956291367057

## Trying Some Artist Recommendations
Using the best model above, predict the top 5 artists for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real artist name using `artistAlias`. Print the results. The output should look as follows:
```
Artist 0: Brand New
Artist 1: Taking Back Sunday
Artist 2: Evanescence
Artist 3: Elliott Smith
Artist 4: blink-182
```

In [14]:
ratings = bestModel.recommendProducts(1059637, 5)
artist_data = dict(artistData.collect())

# While loop implementation to print the 5 artist recommendation for the user
i = 0 
while i < 5:
    suggest_artist = map(lambda x: artist_data[x.product] 
                         if x.product not in artist_alias_info 
                         else artist_data[artist_alias_info[x.product]], ratings)
    print "Artist %d: %s"%(i,str(suggest_artist[i]))
    i += 1

Artist 0: Pink Floyd
Artist 1: Rage Against the Machine
Artist 2: Elliott Smith
Artist 3: Jimmy Eat World
Artist 4: Taking Back Sunday


In [ ]:
# File created by Aditya Bhardwaj
# Unity ID: abhardw2

# Experimented for different ranks 2, 10, 20

# The result of Artist recommendation for rank = 2 is shown above.
# Artist 0: Pink Floyd
# Artist 1: Rage Against the Machine
# Artist 2: Elliott Smith
# Artist 3: Jimmy Eat World
# Artist 4: Taking Back Sunday

# The result of Artist recommendation for rank = 10 is below:
# Artist 0: blink-182
# Artist 1: Elliott Smith
# Artist 2: Taking Back Sunday
# Artist 3: Incubus
# Artist 4: Death Cab for Cutie

# The result of Artist recommendation for rank = 20 is below:
# Artist 0: Taking Back Sunday
# Artist 1: Elliott Smith
# Artist 2: Incubus
# Artist 3: Death Cab for Cutie
# Artist 4: Hanson

